# Bibliotecas

In [ ]:
#Importa bibliotecas necessárias
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import math

---
# Parâmetros do modelo

In [ ]:
# Dimensões (m)
r = 0                                   # Copo (raio interno)
h_copo = 0                              # Copo (altura)
d_copo = 0/2                            # Copo (espessura)/2

# Áreas (m2)
As = math.pi*r**2                       # Água (superfície)
Api = h_copo*2*math.pi*r                # Copo (paredes internas)
Ape = Api                               # Copo (paredes externas)

# Volumes (m3)
v_agua = As*h_copo                      # Água
v_copo = 2*d_copo*(Ape+As)              # Recipiente

# Densidades (kg/m3)
rho_agua = 0                            # Água
rho_copo = 0                            # Copo (vidro)

# Massas (kg)
m_agua = rho_agua*v_agua                # Água
m_copo = rho_copo*v_copo                # Copo

# Calores específicos (J/(kg.K))
c_agua = 0                              # Água
c_copo = 0                              # Copo (vidro)

# Coeficiente de condutividade térmica (W/(m.K))
k_copo = 0                              # Copo (vidro)

# Coeficientes de transferência convectiva (W/(m2.K))
ha = 0                                  # Água ➞ ar
hc = 0                                  # Copo ➞ ar

#Temperaturas (K)
T_ar = 26+273.1                         # Ambiente
T_agua = 90                             # Água
T_copo = 90                             # Copo

---
# Funções de transferência do calor

## Fluxo de calor água ➞ ar

In [ ]:

def Fun_Qagua_ar(T_agua):

    ΔT = T_agua - T_ar
    Qagua_ar = ha * As * ΔT

    return Qagua_ar

----
## Fluxo de calor água ➞ copo

In [ ]:

def Fun_Qagua_copo(T_agua, T_copo):
    
    ΔT = T_agua - T_copo
    Qagua_copo = (k_copo * Api * ΔT) / d_copo

    return Qagua_copo

----
## Fluxo de calor copo ➞ ar

In [ ]:

def Fun_Qcopo_ar(T_copo):

    ΔT = T_copo - T_ar
    Qcopo_ar = ΔT / ((d_copo / (k_copo * Ape)) + (1 / (hc * Ape)))
    
    return Qcopo_ar

---
# Implemetação

In [ ]:

def modelo(x, lista_t_h):

    T_agua = x[0]
    T_copo = x[1]

    dT_aguadt = -Fun_Qagua_ar(T_agua) - Fun_Qagua_copo(T_agua, T_copo) / (m_agua * c_agua)
    dT_copodt = Fun_Qagua_copo(T_agua, T_copo) - Fun_Qcopo_ar(T_copo) / (m_copo * c_copo)

    dxdt = [dT_aguadt, dT_copodt]    

    return dxdt
    

In [ ]:

lista_t_s = np.arange(0, 7200, 20)  # tempo (s)
lista_t_h = lista_t_s/3600          # tempo (h)

x0 = [T_agua, T_copo]

lista_x = odeint(modelo, x0, lista_t_h)

lista_Ta = lista_x[:,0]-273.15
lista_Tc = lista_x[:,1]-273.15

plt.plot(lista_t_h, lista_Ta, label = '$T_água$ (modelo)')
plt.plot(lista_t_h, lista_Tc, label = '$T_copo$ (modelo)')
# plt.plot(lista_t_h, experimento, '$T_água$ (experimento)')

plt.title('Modelo vs Experimento')
plt.xlabel('Tempo (h)')
plt.ylabel('Temperatura (°C)')
plt.legend()
plt.grid(True)
plt.show()